# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_6 MWT', 'verschil_TUG',
               'verschil_BMI', 'verschil_Conditie',
               'verschil_Lenigheid', 'verschil_Gewicht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_Knijpkracht'])
y = df['verschil_Knijpkracht']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['AfstandTotBos', 'ZwaarBelasteMantelzorgers']
Dropped features: ['EenOfMeerLangdurigeAandoeningen', 'AfstandTotOpenDroogNatTerrein', 'GeboorteRelatief', 'Rokers', 'AfstandTotKinderdagverblijf', 'OppervlakteWater', 'ALandbouwBosbouwEnVisserij', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotVolkstuin', 'AfstandTotDagrecreatiefTerrein', 'BeperkingInHoren', 'Mantelzorger', 'AfstandTotZonnebank', 'AfstandTotSchool', 'NederlandseAntillenEnAruba', 'AfstandTotBegraafplaats', 'AfstandTotPoppodium', 'TotaalDiefstalUitWoningSchuurED', 'KLFinancieleDienstenOnroerendGoed', 'AfstandTotBibliotheek', 'AfstandTotOpenbaarGroenTotaal', 'PercentageOnbewoond', 'AfstandTotApotheek', 'BeperkingInZien', 'InBezitWoningcorporatie', 'PersonenautoSOverigeBrandstof', 'AfstandTotSauna', 'AfstandTotOvDagelLevensmiddelen', 'MatigOvergewicht', 'AfstandTotHuisartsenpost', 'AfstandTotTreinstationsTotaal', 'AfstandTotCafetariaED', 'AfstandTotSemiOpenbaarGroenTotaal', 'PersonenautoSPerHuishouden', 'Uren

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------
| 1         | -25.3     | 0.9208    | 0.03075   | 3.208     |
| 2         | -25.24    | 2.441     | 0.03086   | 2.147     |
| 3         | -25.56    | 5.474     | 0.05591   | 2.601     |
| 4         | -25.67    | 9.817     | 0.0448    | 2.346     |
| 5         | -24.83    | 5.624     | 0.005085  | 3.997     |
| 6         | -25.41    | 2.324     | 0.08161   | 2.318     |
| 7         | -25.82    | 4.907     | 0.03956   | 3.93      |
| 8         | -25.79    | 3.821     | 0.08649   | 1.561     |
| 9         | -25.36    | 4.086     | 0.03134   | 2.823     |
| 10        | -25.11    | 6.129     | 0.003914  | 2.141     |
| 11        | -25.03    | 0.6918    | 0.01933   | 3.28      |
| 12        | -25.68    | 3.293     | 0.03275   | 3.121     |
| 13        | -25.86    | 5.825     | 0.04978   | 3.983     |
| 14        | -25.86    | 5.924     | 0.06652   | 1.246     |
| 15    

SVR(C=5.624373971710246, gamma=0.005085470973802771, kernel='poly')

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['ALandbouwBosbouwEnVisserij']
Dropped features: ['EenOfMeerLangdurigeAandoeningen', 'AfstandTotOpenDroogNatTerrein', 'GeboorteRelatief', 'Rokers', 'AfstandTotKinderdagverblijf', 'OppervlakteWater', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotVolkstuin', 'AfstandTotDagrecreatiefTerrein', 'BeperkingInHoren', 'Mantelzorger', 'AfstandTotZonnebank', 'AfstandTotSchool', 'NederlandseAntillenEnAruba', 'AfstandTotBegraafplaats', 'AfstandTotPoppodium', 'TotaalDiefstalUitWoningSchuurED', 'KLFinancieleDienstenOnroerendGoed', 'AfstandTotBibliotheek', 'AfstandTotOpenbaarGroenTotaal', 'PercentageOnbewoond', 'AfstandTotApotheek', 'BeperkingInZien', 'InBezitWoningcorporatie', 'PersonenautoSOverigeBrandstof', 'AfstandTotSauna', 'AfstandTotOvDagelLevensmiddelen', 'MatigOvergewicht', 'AfstandTotHuisartsenpost', 'AfstandTotTreinstationsTotaal', 'AfstandTotCafetariaED', 'AfstandTotSemiOpenbaarGroenTotaal', 'PersonenautoSPerHuishouden', 'UrenMantelzorgPerWeek', 'AfstandTotBioscoop', 'Afs

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -24.93    | 12.69     | 0.991     | 11.36     | 761.1     |
| 2         | -24.93    | 19.25     | 0.5006    | 2.37      | 534.6     |
| 3         | -24.93    | 12.88     | 0.5632    | 2.749     | 383.0     |
| 4         | -24.93    | 13.1      | 0.3733    | 11.54     | 762.6     |
| 5         | -24.97    | 4.032     | 0.2173    | 10.38     | 728.2     |
| 6         | -24.93    | 8.277     | 0.8029    | 8.257     | 481.7     |
| 7         | -24.93    | 19.78     | 0.1907    | 9.966     | 218.6     |
| 8         | -24.93    | 12.47     | 0.7779    | 9.25      | 600.4     |
| 9         | -24.88    | 2.914     | 0.4019    | 7.635     | 422.5     |
| 10        | -24.93    | 10.35     | 0.5521    | 10.22     | 457.4     |
| 11        | -24.93    | 15.07     | 0.9141    | 2.35      | 420.2     |
| 12        | -24.88    | 2.805     | 

RandomForestRegressor(bootstrap=False, max_depth=2,
                      max_features=0.14786307819947042, min_samples_split=9,
                      n_estimators=447)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['GeweldsEnSeksueleMisdrijven']
Dropped features: ['EenOfMeerLangdurigeAandoeningen', 'AfstandTotOpenDroogNatTerrein', 'GeboorteRelatief', 'Rokers', 'AfstandTotKinderdagverblijf', 'OppervlakteWater', 'ALandbouwBosbouwEnVisserij', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotVolkstuin', 'AfstandTotDagrecreatiefTerrein', 'BeperkingInHoren', 'Mantelzorger', 'AfstandTotZonnebank', 'AfstandTotSchool', 'NederlandseAntillenEnAruba', 'AfstandTotBegraafplaats', 'AfstandTotPoppodium', 'TotaalDiefstalUitWoningSchuurED', 'KLFinancieleDienstenOnroerendGoed', 'AfstandTotBibliotheek', 'AfstandTotOpenbaarGroenTotaal', 'PercentageOnbewoond', 'AfstandTotApotheek', 'BeperkingInZien', 'InBezitWoningcorporatie', 'PersonenautoSOverigeBrandstof', 'AfstandTotSauna', 'AfstandTotOvDagelLevensmiddelen', 'MatigOvergewicht', 'AfstandTotHuisartsenpost', 'AfstandTotTreinstationsTotaal', 'AfstandTotCafetariaED', 'AfstandTotSemiOpenbaarGroenTotaal', 'PersonenautoSPerHuishouden', 'UrenMantelzorgPerWe

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -4.95     | 0.01741   | 91.09     | 0.02227   | 722.1     |
| 2         | -4.951    | 0.01562   | 49.69     | 0.02611   | 746.6     |
| 3         | -4.946    | 0.06453   | 88.25     | 0.01337   | 605.6     |
| 4         | -4.956    | 0.02161   | 39.27     | 0.08851   | 416.5     |
| 5         | -4.942    | 0.07996   | 37.3      | 0.09379   | 654.6     |
| 6         | -4.952    | 0.03324   | 58.58     | 0.05339   | 555.2     |
| 7         | -4.943    | 0.0515    | 77.05     | 0.04451   | 225.1     |
| 8         | -4.949    | 0.05207   | 43.79     | 0.07301   | 698.5     |
| 9         | -4.943    | 0.034     | 54.94     | 0.008509  | 975.6     |
| 10        | -4.947    | 0.03859   | 87.31     | 0.04347   | 780.8     |
| 11        | -4.939    | 0.1       | 10.0      | 0.1       | 629.9     |
| 12        | -4.946    | 0.03709   | 

MLPRegressor(alpha=0.00717303702078568, hidden_layer_sizes=(57,),
             learning_rate_init=0.09555448081663771, max_iter=979)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 3.5479060457091083,
  'mse': 24.72193256519051,
  'rmse': 4.972115501996158,
  'r2': 0.0001641782064960573,
  'adj_r2': -0.0770006591976331},
 {'model': 'Random Forest',
  'mae': 3.5679871211987404,
  'mse': 24.914171126270848,
  'rmse': 4.991409733358989,
  'r2': -0.007610578042487104,
  'adj_r2': -0.0853754517613119},
 {'model': 'Neural Network',
  'mae': 3.5790822107970026,
  'mse': 24.94288192938588,
  'rmse': 4.994284926732343,
  'r2': -0.008771736837463129,
  'adj_r2': -0.08662622589878888},
 {'model': 'Support Vector Machine',
  'mae': 3.543125897532928,
  'mse': 24.84806393849684,
  'rmse': 4.984783238867748,
  'r2': -0.004936987123161263,
  'adj_r2': -0.08249551975670188}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        text=metric_scores[metric],  # Display the values on the bars
        textposition='auto',  # Position the text inside the bars
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 3.6547330972736276,
  'mse': 25.668758181814894,
  'rmse': 5.066434464375799,
  'r2': 0.002311251464875541,
  'adj_r2': -0.39905778530212843},
 {'model': 'Random Forest',
  'mae': 3.693346296531759,
  'mse': 26.027548531026813,
  'rmse': 5.101720154127117,
  'r2': -0.011634148306949754,
  'adj_r2': -0.41861340337296404},
 {'model': 'Neural Network',
  'mae': 3.6558830491010523,
  'mse': 25.86057730792518,
  'rmse': 5.085329616448198,
  'r2': -0.005144340368532729,
  'adj_r2': -0.4095127531604712},
 {'model': 'Support Vector Machine',
  'mae': 3.656931401427199,
  'mse': 25.6584615767374,
  'rmse': 5.0654182035383215,
  'r2': 0.002711458088085039,
  'adj_r2': -0.39849657601440946}]

In [18]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        text=metric_scores[metric],  # Display the values on the bars
        textposition='auto',  # Position the text inside the bars
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison Knijpkracht',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 3.6518080074565615
Mean Squared Error: 25.781358746682066
Root Mean Squared Error: 5.077534711519171
R2 Score: -0.0020652873552322593
Adjusted R2 Score: -0.40519500065906144


Save best model as Pickle

In [17]:
import pickle

# Save model
file_path = 'Saved Models/Knijpkracht.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_Knijpkracht.txt', 'w') as file:
    file.write(f'{best_features_mlr}')